In [38]:
import os

In [39]:
os.getcwd()

'E:\\TSVD'

In [40]:
%cd E:

E:\TSVD


In [41]:
%ls

 Volume in drive E is ubuntu
 Volume Serial Number is 1E3C-9B9B

 Directory of E:\TSVD

01/31/2025  06:14 PM    <DIR>          .
01/27/2025  07:47 PM    <DIR>          .ipynb_checkpoints
01/28/2025  09:14 PM    <DIR>          __pycache__
01/28/2025  08:00 PM             3,278 animegan.py
01/28/2025  08:47 PM    <DIR>          AnimeGANv2
02/01/2025  07:35 PM            84,864 checkpoints.log
11/30/2024  09:20 PM        23,269,826 dance1.mp4
01/31/2025  06:00 PM             3,045 enime.py.txt
01/29/2025  08:21 AM             1,910 file2.py.txt
01/31/2025  05:06 PM    <DIR>          frames
01/28/2025  11:32 AM        46,296,035 Hayao.tar.gz
01/29/2025  10:31 PM                 1 last_frame_count.txt
01/28/2025  08:42 PM               282 main.py
01/28/2025  08:08 PM    <DIR>          models
02/01/2025  07:35 PM    <DIR>          temp_frames
01/30/2025  11:00 PM     1,634,275,817 temp_frames.zip
01/28/2025  11:10 AM             4,381 train.py
01/27/2025  07:10 PM        59,897,969 TSVD1.mp

In [5]:
%cd TSVD

E:\TSVD


In [6]:
%ls

 Volume in drive E is ubuntu
 Volume Serial Number is 1E3C-9B9B

 Directory of E:\TSVD

01/31/2025  06:14 PM    <DIR>          .
01/27/2025  07:47 PM    <DIR>          .ipynb_checkpoints
01/28/2025  09:14 PM    <DIR>          __pycache__
01/28/2025  08:00 PM             3,278 animegan.py
01/28/2025  08:47 PM    <DIR>          AnimeGANv2
02/01/2025  02:29 PM            69,840 checkpoints.log
11/30/2024  09:20 PM        23,269,826 dance1.mp4
01/31/2025  06:00 PM             3,045 enime.py.txt
01/29/2025  08:21 AM             1,910 file2.py.txt
01/31/2025  05:06 PM    <DIR>          frames
01/28/2025  11:32 AM        46,296,035 Hayao.tar.gz
01/29/2025  10:31 PM                 1 last_frame_count.txt
01/28/2025  08:42 PM               282 main.py
01/28/2025  08:08 PM    <DIR>          models
02/01/2025  01:18 PM    <DIR>          temp_frames
01/30/2025  11:00 PM     1,634,275,817 temp_frames.zip
01/28/2025  11:10 AM             4,381 train.py
01/27/2025  07:10 PM        59,897,969 TSVD1.mp

In [7]:
import psutil

# Get memory information
memory_info = psutil.virtual_memory()
print(f"Total Memory: {memory_info.total / (1024 ** 3):.2f} GB")
print(f"Available Memory: {memory_info.available / (1024 ** 3):.2f} GB")
print(f"Used Memory: {memory_info.used / (1024 ** 3):.2f} GB")
print(f"Memory Usage: {memory_info.percent}%")

# Get disk information
disk_info = psutil.disk_usage('/')
print(f"Total Disk Space: {disk_info.total / (1024 ** 3):.2f} GB")
print(f"Used Disk Space: {disk_info.used / (1024 ** 3):.2f} GB")
print(f"Free Disk Space: {disk_info.free / (1024 ** 3):.2f} GB")
print(f"Disk Usage: {disk_info.percent}%")

Total Memory: 15.41 GB
Available Memory: 10.09 GB
Used Memory: 5.31 GB
Memory Usage: 34.5%
Total Disk Space: 156.25 GB
Used Disk Space: 11.11 GB
Free Disk Space: 145.14 GB
Disk Usage: 7.1%


%cd AnimeGANv2

!pip install -r requirements.txt

In [8]:
import os
import cv2
import numpy as np
import onnxruntime as ort
import typing

# Set the environment variable to use the first NVIDIA GPU
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

class AnimeGAN:
    def __init__(self, model_path: str, downsize_ratio: float = 1.0) -> None:
        """
        Args:
            model_path: (str) - path to onnx model file
            downsize_ratio: (float) - ratio to downsize input frame for faster inference
                onnx models:
        'https://docs.google.com/uc?export=download&id=1VPAPI84qaPUCHKHJLHiMK7BP_JE66xNe' AnimeGAN_Hayao.onnx
        'https://docs.google.com/uc?export=download&id=17XRNQgQoUAnu6SM5VgBuhqSBO4UAVNI1' AnimeGANv2_Hayao.onnx
        'https://docs.google.com/uc?export=download&id=10rQfe4obW0dkNtsQuWg-szC4diBzYFXK' AnimeGANv2_Shinkai.onnx
        'https://docs.google.com/uc?export=download&id=1X3Glf69Ter_n2Tj6p81VpGKx7U4Dq-tI' AnimeGANv2_Paprika.onnx

        """
        if not os.path.exists(model_path):
            raise Exception(f"Model doesn't exist at {model_path}")

        self.downsize_ratio = downsize_ratio

        # Check for CUDA availability and set providers
        available_providers = ort.get_available_providers()
        if "CUDAExecutionProvider" in available_providers:
            self.providers = ['CUDAExecutionProvider']
        else:
            self.providers = ['CPUExecutionProvider']

        self.ort_sess = ort.InferenceSession(model_path, providers=self.providers)

    def to_32s(self, x):
        """ Ensure input is multiple of 32 """
        return 256 if x < 256 else x - x % 32

    def process_frame(self, frame: np.ndarray, x32: bool = True) -> np.ndarray:
        """ Function to process frame to fit model input as 32 multiplier and resize to fit model input """
        h, w = frame.shape[:2]
        if x32:  # resize image to multiple of 32s
            frame = cv2.resize(frame, (self.to_32s(int(w * self.downsize_ratio * 2)), self.to_32s(int(h * self.downsize_ratio * 2))))
        frame = frame.astype(np.float32) / 127.5 - 1.0
        return frame

    def post_process(self, frame: np.ndarray, wh: typing.Tuple[int, int]) -> np.ndarray:
        """ Convert model float output to uint8 image resized to double the original frame size """
        frame = (frame.squeeze() + 1.) / 2 * 255
        frame = frame.astype(np.uint8)
        frame = cv2.resize(frame, (wh[0] * 2, wh[1] * 2))
        return frame

    def __call__(self, frame: np.ndarray) -> np.ndarray:
        """ Main function to process each frame """
        image = self.process_frame(frame)

        # Get input name
        input_name = self.ort_sess.get_inputs()[0].name

        # Run inference
        outputs = self.ort_sess.run(None, {input_name: np.expand_dims(image, axis=0)})

        # Post-process the output
        frame = self.post_process(outputs[0], frame.shape[:2][::-1])
        return frame

!pip install --upgrade onnxruntime-gpu


In [9]:
ort.get_available_providers()

['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']

In [10]:
import onnxruntime as ort
print(ort.get_available_providers())
print("ONNX Runtime version:", ort.__version__)

['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
ONNX Runtime version: 1.19.2


In [11]:
session = ort.InferenceSession("models/Shinkai_53.onnx", providers=['CUDAExecutionProvider'])

In [12]:
num_gpus = len(session.get_providers())
print("Number of GPUs available:", num_gpus)

Number of GPUs available: 1


In [ ]:
import cv2
import os

# Function to split video into frames
def split_video_to_frames(video_path, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Save the frame as an image file
        frame_filename = os.path.join(output_folder, f'frame_{frame_count:04d}.jpg')
        cv2.imwrite(frame_filename, frame)

        frame_count += 1

    # Release the video capture object
    cap.release()
    print(f"Video split into {frame_count} frames and saved in '{output_folder}'.")

# Example usage
video_name = 'dance1'
video_path = f'{video_name}.mp4'
output_folder = 'frames'
split_video_to_frames(video_path, output_folder)

In [26]:
# Example usage
video_name = 'dance1'
video_path = f'{video_name}.mp4'
output_folder = 'frames'

In [14]:
import os

# Get a list of all files in temp_frames
files_in_temp_frames = [f for f in os.listdir('temp_frames') if os.path.isfile(os.path.join('temp_frames', f))]

# Append each filename to checkpoints.log
with open('checkpoints.log', 'a') as f:
    for filename in files_in_temp_frames:
        f.write(f"{filename}\n")

print(f"Inserted {len(files_in_temp_frames)} filenames into checkpoints.log")

def order_and_deduplicate_checkpoint_names(checkpoint_file):
    """Orders and deduplicates checkpoint names in the given file.

    Args:
        checkpoint_file: The path to the checkpoint file.
    """
    with open(checkpoint_file, 'r') as f:
        checkpoint_names = f.read().splitlines()

    # Remove empty lines
    checkpoint_names = [name for name in checkpoint_names if name]

    # Order the names
    checkpoint_names.sort()

    # Deduplicate the names
    unique_checkpoint_names = []
    for name in checkpoint_names:
        if name not in unique_checkpoint_names:
            unique_checkpoint_names.append(name)

    # Write the unique names back to the file
    with open(checkpoint_file, 'w') as f:
        for name in unique_checkpoint_names:
            f.write(f"{name}\n")

    print(f"Ordered and deduplicated checkpoint names in '{checkpoint_file}'.")

# Call the function with the path to your checkpoint file
checkpoint_file = 'checkpoints.log'
order_and_deduplicate_checkpoint_names(checkpoint_file)

Inserted 4365 filenames into checkpoints.log
Ordered and deduplicated checkpoint names in 'checkpoints.log'.


In [15]:
def get_sorted_files_from_folder(folder):
    """Retrieve and sort file names from a specified folder."""
    files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
    return sorted(files)

# Get and print sorted files from temp_frames and frames
sorted_temp_frames_files = get_sorted_files_from_folder('temp_frames')
sorted_frames_files = get_sorted_files_from_folder('frames')

In [16]:
# Initialize the AnimeGAN model with GPU1 support
model_path = "models/Shinkai_53.onnx"
#animegan = AnimeGAN(model_path, device_id=1)
animegan = AnimeGAN(model_path)

In [17]:
# Example usage
input_folder = 'frames'
output_folder = 'temp_frames'
checkpoint_file = 'checkpoints.log'

In [42]:
 # Load the checkpoint log and get the list of already processed files
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        processed_files = f.read().splitlines()
else:
    processed_files = []

In [43]:
len(processed_files)

5304

In [20]:
%%time
import os
import cv2
import time

# Function to process images in a folder and save to a new folder with the same filenames
def process_images(input_folder, output_folder, checkpoint_file):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Load the checkpoint log and get the list of already processed files
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            processed_files = f.read().splitlines()
    else:
        processed_files = []

    # List all files in the input folder
    input_files = sorted([f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))])

    # Process only files that exist in the checkpoint log
    for input_file in input_files:
        # Start time
        start_time = time.time()
        if input_file not in processed_files:
            # Read the image
            input_path = os.path.join(input_folder, input_file)
            frame = cv2.imread(input_path)

            # Process the image with AnimeGAN
            processed_frame = animegan(frame)

            # Save the processed image to the output folder with the same filename
            output_path = os.path.join(output_folder, input_file)
            cv2.imwrite(output_path, processed_frame)

            # Update the checkpoint log
            with open(checkpoint_file, 'a') as f:
                f.write(f"{input_file}\n")
            
            # End time
            end_time = time.time()
            print(f"Processed '{input_file}' and saved to '{output_folder}'.")
            # Calculate duration
            duration = end_time - start_time
            print(f"Inference took {duration:.2f} seconds")

# Example usage
input_folder = 'frames'
output_folder = 'temp_frames'
checkpoint_file = 'checkpoints.log'
process_images(input_folder, output_folder, checkpoint_file)

Processed 'frame_4365.jpg' and saved to 'temp_frames'.
Inference took 17.39 seconds
Processed 'frame_4366.jpg' and saved to 'temp_frames'.
Inference took 16.97 seconds
Processed 'frame_4367.jpg' and saved to 'temp_frames'.
Inference took 16.71 seconds
Processed 'frame_4368.jpg' and saved to 'temp_frames'.
Inference took 15.95 seconds
Processed 'frame_4369.jpg' and saved to 'temp_frames'.
Inference took 16.49 seconds
Processed 'frame_4370.jpg' and saved to 'temp_frames'.
Inference took 16.23 seconds
Processed 'frame_4371.jpg' and saved to 'temp_frames'.
Inference took 16.12 seconds
Processed 'frame_4372.jpg' and saved to 'temp_frames'.
Inference took 16.31 seconds
Processed 'frame_4373.jpg' and saved to 'temp_frames'.
Inference took 16.74 seconds
Processed 'frame_4374.jpg' and saved to 'temp_frames'.
Inference took 16.37 seconds
Processed 'frame_4375.jpg' and saved to 'temp_frames'.
Inference took 16.39 seconds
Processed 'frame_4376.jpg' and saved to 'temp_frames'.
Inference took 16.35 

In [49]:
import cv2
import os
from moviepy import VideoFileClip

# Function to merge images into a video
def merge_images_to_video(input_folder, output_video_path, frame_rate, frame_width, frame_height):
    # List all files in the input folder and sort them by name (or modify the sorting logic as necessary)
    input_files = sorted([f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png', '.jpeg'))])

    if not input_files:
        print("No image files found in the input folder.")
        return

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can choose 'H264' if needed
    out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (frame_width, frame_height))

    for input_file in input_files:
        input_path = os.path.join(input_folder, input_file)
        frame = cv2.imread(input_path)

        # Ensure the frame is valid
        if frame is None:
            print(f"Error reading {input_file}. Skipping.")
            continue

        # Resize the frame to match the video dimensions
        frame_resized = cv2.resize(frame, (frame_width, frame_height))

        # Write the resized frame to the video
        out.write(frame_resized)

    # Release the video writer
    out.release()
    print(f"Video created and saved to '{output_video_path}'.")

# Set the video name
video_name = 'dance1'

# Paths for input and output
video_path = f'{video_name}.mp4'  # Original video path
output_video_path = f'{video_name}_output.mp4'  # New video after merging images

# Get properties from the original video
cap = cv2.VideoCapture(video_path)  # Open the original video to extract properties
frame_rate = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()  # Release after getting properties

# Folder containing frames to merge into video
input_folder = 'temp_frames'  # Make sure this folder contains the frames you want to merge

# Merge images into video
merge_images_to_video(input_folder, output_video_path, frame_rate, frame_width, frame_height)

# Now, using moviepy to add the audio from the original video
final_video_path = f'{video_name}_final_with_audio.mp4'  # Final output video path

# Load original video to extract audio
original_video = VideoFileClip(video_path)
original_audio = original_video.audio

# Load the newly created video
new_video = VideoFileClip(output_video_path)

# Set the original audio to the new video
final_video = new_video.with_audio(original_audio)

# Write the final video to a file with audio; 8000k (which is 8 Mbps). This is a good middle ground for Full HD (1080p) video quality
final_video.write_videofile(final_video_path, codec='libx264', audio_codec='aac', bitrate='8000k')

print(f"Final video created and saved to '{final_video_path}'.")

Video created and saved to 'dance1_output.mp4'.
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2avc1mp41'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [720, 900], 'bitrate': 788, 'fps': 23.976023976023978, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]', 'encoder': 'AVC Coding'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 48, 'metadata': {'Metadata': '', 'handler_name': 'SoundHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 221.32999999999998, 'bitrate': 841, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(High)', 'video_size': [720, 900], 'video_b

MoviePy - Done.
MoviePy - Writing video dance1_final_with_audio.mp4



MoviePy - Done !
MoviePy - video ready dance1_final_with_audio.mp4
Final video created and saved to 'dance1_final_with_audio.mp4'.
